<a href="https://colab.research.google.com/github/HarshalPawar88/Internship/blob/main/Webscraping_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')`
`

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,428 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://security

In [ ]:
#importing all the required libraries
import selenium
from selenium import webdriver
import time
import requests
import json
from bs4 import BeautifulSoup
import os
from PIL import Image
import io
import hashlib
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException,ElementNotInteractableException,InvalidArgumentException,InvalidCookieDomainException

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [ ]:
#connecting to the webdriver
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [ ]:
#specifying the url of the webpage to be scraped
url="https://www.amazon.in/"

In [ ]:
#opening the webpage through our driver
driver.get(url)

In [ ]:
# Taking input from user 
search_string = input("Input the string you want to search for:")

#generate a url from search string
url= driver.get("https://www.amazon.in/s?k=" + search_string + "&start=")

Input the string you want to search for:Laptop


### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [ ]:
#getting the list of url of televisions
Laptop_URL=[]

for r in range(0,3):
    laptop=driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']")
    for i in laptop:
        laptop_2=i.find_element_by_tag_name("a")
        Laptop_URL.append(laptop_2.get_attribute('href'))
try:        
    driver.find_element_by_xpath('//*[@id="search"]/div[1]/div[2]/div/span[3]/div[2]/div[27]/span/div/div/ul/li[3]/a').click()
except NoSuchElementException:
  pass

In [ ]:
Laptop_URL

In [ ]:
#checking the length of tv url
len(Laptop_URL)

66

In [ ]:
#going to the url of first television through our driver
driver.get(Laptop_URL[0])

In [ ]:
#creating the empty lists
Brand_name=[]
Name_of_product=[]
Rating=[]
No_of_ratings=[]
Price=[]
Return_exchange=[]
Expected_delivery=[]
Availibility=[]
Other_details=[]

In [ ]:
#running a loop to extract all the required information from Laptop_Url

from tqdm import tqdm
for i in tqdm(Laptop_URL):
    driver.get(i)
    try:
        brand=driver.find_element_by_xpath('//*[@id="bylineInfo"]').text
        Brand_name.append(brand)
    except NoSuchElementException:
        Brand_name.append('-')
        
    try:
        product=driver.find_element_by_xpath('//*[@id="productTitle"]').text
        Name_of_product.append(product)
    except NoSuchElementException:
        Name_of_product.append('-')
    
    try:
        rating_=driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']").click()
        rating=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-base']").text
        Rating.append(rating)
    except NoSuchElementException as e:
        Rating.append('-')
        
    try:
        ratings=driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]').text
        No_of_ratings.append(ratings)
    except NoSuchElementException:
        No_of_ratings.append('-')
        
    try:
        price=driver.find_element_by_xpath('//*[@id="priceblock_ourprice"]').text
        Price.append(price)
    except NoSuchElementException:
        Price.append('-')
        
    try:
        exchange=driver.find_element_by_xpath('//*[@id="icon-farm-container"]/div/div[1]/span/div[2]/a').text
        Return_exchange.append(exchange)
    except NoSuchElementException:
        Return_exchange.append('-')
        
    try:
        delivery=driver.find_element_by_xpath('//*[@id="ddmDeliveryMessage"]/b').text
        Expected_delivery.append(delivery)
    except NoSuchElementException:
        Expected_delivery.append('-')
        
    try:
        availibility=driver.find_element_by_xpath('//*[@id="availability"]/span').text
        Availibility.append(availibility)
    except NoSuchElementException:
        Availibility.append('-')
        
    try:
        details=driver.find_element_by_xpath('//*[@id="feature-bullets"]/ul').text.replace("\n"," ")
        Other_details.append(details)
    except NoSuchElementException:
        Other_details.append('-')
        
    time.sleep(3)

100%|██████████| 66/66 [07:28<00:00,  6.80s/it]


In [ ]:
Brand_name

In [ ]:
Name_of_product

In [ ]:
Rating

In [ ]:
No_of_ratings

In [ ]:
Price

In [ ]:
Return_exchange

In [ ]:
Expected_delivery

In [ ]:
Availibility

In [ ]:
Other_details

In [ ]:
#printing length
print(len(Brand_name),len(Name_of_product),len(Rating),len(No_of_ratings),len(Price),len(Return_exchange),len(Expected_delivery),len(Availibility),len(Other_details),len(Laptop_URL))

66 66 66 66 66 66 66 66 66 66


In [ ]:
#creating a dataframe

Laptops=pd.DataFrame({})
Laptops['Brand_name']=Brand_name
Laptops['Name_of_product']=Name_of_product
Laptops['Rating']=Rating
Laptops['No_of_ratings']=No_of_ratings
Laptops['Price']=Price
Laptops['Return_exchange']=Return_exchange
Laptops['Expected_delivery']=Expected_delivery
Laptops['Availibility']=Availibility
Laptops['Other_details']=Other_details
Laptops['Product_URL']=Laptop_URL

In [ ]:
Laptops

,Brand_name,Name_of_product,Rating,No_of_ratings,Price,Return_exchange,Expected_delivery,Availibility,Other_details,Product_URL
0,Visit the HP Store,HP Pavilion 14 11th Gen Intel Core i5 14-inch ...,4.4 out of 5,"1,032 ratings","₹62,990.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,Processor: 11th Gen Intel Core i5-1135G7 (up t...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Visit the HP Store,HP Chromebook 14-inch (35.56 cms) Thin & Light...,3.8 out of 5,774 ratings,"₹27,990.00",-,"Tuesday, Sep 28",In stock.,GOOGLE ASSISTANT: Voice-Enabled Google Assista...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Visit the Acer Store,Acer Aspire 3 AMD Athlon Silver 3050U Dual-cor...,3.5 out of 5,4 ratings,"₹32,500.00",7 Days Replacement,"Wednesday, Sep 29",In stock.,Laptops Processor : AMD Athlon Silver 3050U du...,https://www.amazon.in/Acer-Aspire-Athlon-Dual-...
3,Visit the Dell Store,"Dell Inspiron 3502 15.6"" (39.62 cms) HD Displa...",4.2 out of 5,225 ratings,"₹40,126.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,Free upgrade to Windows 11 when available. Dis...,https://www.amazon.in/Dell-Inspiron-Integrated...
4,Visit the HP Store,HP Chromebook 14-inch (35.56 cms) Thin & Light...,3.8 out of 5,774 ratings,"₹27,990.00",-,"Tuesday, Sep 28",In stock.,GOOGLE ASSISTANT: Voice-Enabled Google Assista...,https://www.amazon.in/HP-Chromebook-14a-na0003...
...,...,...,...,...,...,...,...,...,...,...
61,Visit the HP Store,HP 15 11th Gen Intel Core i3 Processor 15.6 in...,4.2 out of 5,"1,023 ratings","₹46,999.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,Free upgrade to Windows 11 when available* Dis...,https://www.amazon.in/HP-Processor-39-62cms-In...
62,Visit the HP Store,HP 15 (2021) Thin & Light 11th Gen Core i3 Lap...,3.9 out of 5,"1,120 ratings","₹41,990.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,Processor: 11th Gen Intel Core i3-1115G4 (up t...,https://www.amazon.in/HP-Laptop-15-6-inch-Wind...
63,Visit the ASUS Store,"ASUS TUF Gaming F17 (2021), 17.3-inch (43.94 c...",4.2 out of 5,3 ratings,"₹1,13,990.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,Free upgrade to Windows 11 when available. Dis...,https://www.amazon.in/ASUS-17-3-inch-i7-11800H...
64,Visit the Lenovo Store,Lenovo IdeaPad Slim 3 Intel Core i5 10th Gen 1...,3.8 out of 5,53 ratings,"₹64,090.00",7 Days Replacement,"Thursday, Oct 7",Only 1 left in stock.,Processor: Intel Core i5 10th Generation (i5-1...,https://www.amazon.in/gp/slredirect/picassoRed...


In [ ]:
#converting into csv format
Laptops.to_csv("Laptops.csv")

In [ ]:
#closing the driver
driver.close()

### 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [ ]:

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
webd= webdriver.Chrome('chromedriver',chrome_options=chrome_options)
#path
def get_urls(query, max_links_to_fetch:100, webd:webd, delays:int=1):
    def END(webd):
        webd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(delays)    
    # load the page
    webd.get("https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img".format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        END(webd)

        # get all image thumbnail results
        thumbnail_results = webd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(delays)
            except Exception:
                continue

            # extract image urls    
            actual_images = webd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = webd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                webd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

if __name__ == '__main__':
    #wd = webdriver.Chrome(executable_path=driver)
    queries = ["fruits","machine learning","cars"]  #set of querries
    for query in queries:
        webd.get('https://google.com')
        search_box = webd.find_element_by_css_selector('input.gLFyf')
        search_box.send_keys(query)
        links = get_urls(query,100,webd)
        #give images path
        images_path = r'/content/drive/MyDrive/Images'
        for i in links:
            persist_image(images_path,query,i)
    webd.quit()

Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
SUCCESS - saved https://www.treehugger.com/thmb/G0py-g_79QMjUDpE3SqnuD1ZWYo=/1000x562/smart/filters:no_upscale()/__opt__aboutcom__coeus__resources__content_migration__mnn__images__2017__01__Collection-Exotic-Fruits-Wooden-Table-808a6d28dd704338aa8012c5f335a7e7.jpg - as /content/drive/MyDrive/Images/fruits/4994c5442b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTDK35D457WbhLPKY1ahl43QAy4JGf4s4BKzQ&usqp=CAU - as /content/drive/MyDrive/Images/fruits/ead94c641d.jpg
SUCCESS - saved https://static.scientificamerican.com/sciam/cache/file/528E0B49-CDD0-42D4-B5BAA3EBAEC01AE6_source.jpg - as /content/drive/MyDrive/Images/fruits/8e847422fa.jpg
SUCCESS - saved https://thumbor.thedailymeal.com/Bm3fFYuigHsPzID1JpnxP2uk5ks=//https://www.thedailymeal.com/sites/default/files/story/2018/iStock-636877252.jpg - as /content/drive/MyDrive/Images/fruits/771362ab91.jpg
SUCCESS - saved https://www.

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


SUCCESS - saved https://pragaglobal.com/wp-content/themes/praga/assets/img/home/r1-home.png - as /content/drive/MyDrive/Images/cars/d9eb180251.jpg
SUCCESS - saved https://cdn.motor1.com/images/mgl/Q0gy8/s1/2020-new-models-guide-lead.webp - as /content/drive/MyDrive/Images/cars/f07848368e.jpg
SUCCESS - saved https://media.wired.com/photos/5fd7ff52e0d489b7b9fd2e33/1:1/w_2400,c_limit/Business-MustangE-ERA04140.jpg - as /content/drive/MyDrive/Images/cars/b481272b3c.jpg
SUCCESS - saved https://www.motortrend.com/uploads/sites/5/2019/12/MotorTrend-Most-Important-Cars-of-the-Decade.jpg - as /content/drive/MyDrive/Images/cars/9be067e6ee.jpg
SUCCESS - saved https://platform.cstatic-images.com/in/v2/stock_photos/098efaf1-e69d-41d1-bb0f-48dcbf595942/975893da-31c1-4ce7-95a2-d3af46d3b908.png - as /content/drive/MyDrive/Images/cars/6a76f42f88.jpg
SUCCESS - saved https://www.cars.com/images/trending_content/ford-mustang-mach-e-2021-02-angle--dynamic--exterior--front--red--wheel-oem-65f5ec1a292580937d

### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page.Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [ ]:
#connecting to the webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver= webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
#specifying the url of the webpage to be scraped
url="https://www.flipkart.com/"

In [ ]:
#opening the webpage through our driver
driver.get(url)

In [ ]:
# Taking input from user 
search_string = input("Input the string you want to search for:")

#generate a url from search string
url= driver.get("https://www.flipkart.com/search?q="+search_string+"&as=on&as-show=on&otracker=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&otracker1=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&as-pos=4&as-type=RECENT&suggestionId=samsung+m31&requestId=5c8b7098-e268-4146-b626-2617dd872a64&as-searchtext=samsung%20")

Input the string you want to search for:Asus Smartphone


In [ ]:
#getting the list of url of samsung m31
Phone_URL=[]


phone=driver.find_elements_by_xpath("//div[@class='_2kHMtA']")
for i in phone:
    phone1=i.find_element_by_tag_name("a")
    Phone_URL.append(phone1.get_attribute('href'))

In [ ]:
Phone_URL

['https://www.flipkart.com/asus-rog-phone-5-black-128-gb/p/itm8e381f6b8521f?pid=MOBGYSZM3YT4ZEGZ&lid=LSTMOBGYSZM3YT4ZEGZ1EAFGM&marketplace=FLIPKART&q=Asus+Smartphone&store=tyy%2F4io&srno=s_1_1&otracker=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&otracker1=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&fm=organic&iid=00dd583a-f3bf-482c-ae8d-86a75c6380af.MOBGYSZM3YT4ZEGZ.SEARCH&ppt=None&ppn=None&ssid=muymdqkfbk0000001632738493955&qH=c85b16aa0cf82993',
 'https://www.flipkart.com/asus-rog-phone-3-black-128-gb/p/itmf8623d755871d?pid=MOBFUXPBV3TFMPAH&lid=LSTMOBFUXPBV3TFMPAH63D3RA&marketplace=FLIPKART&q=Asus+Smartphone&store=tyy%2F4io&srno=s_1_2&otracker=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&otracker1=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&fm=organic&iid=00dd583a-f3bf-482c-ae8d-86a75c6380af.MOBFUXPBV3TFMPAH.SEARCH&ppt=None&ppn=None&ssid=muymdqkfbk0000001632738493955&qH=c85b16aa0cf82993',
 'https://www.flipkart.com/asus-rog-phone-3-black-128-gb/p/itm93ba84fa7cce9?pid=MOBFTHGZEYPWHGHE&lid=LSTMOBFTH

In [ ]:
colour=[]
driver.get(Phone_URL[0])
driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
battery=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[8]/table/tbody/tr[1]/td[2]").text
battery
# for i in Colour:
#   colour.append(i.text) /tbody/tr[2]/td[2]/ul/li

'6000 mAh'

In [ ]:
#checking the length
len(Phone_URL)

24

In [ ]:
#going to the url of first phone through our driver
driver.get(Phone_URL[0])

In [ ]:
#creating empty lists

brand_name=[]
smartphone_name=[]
colour=[]
ram=[]
rom=[]
primary_camera=[]
secondary_camera=[]
display_size=[]
display_resolution=[]
processor=[]
processor_cores=[]
battery_capacity=[]
price=[]

In [ ]:
#running a loop to extract all the required information from the Phone_URL
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
for i in tqdm(Phone_URL):
    driver.get(i)
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
    
    #try:
    #driver.find_element_by_link_text('Read More').click()
    #except NoSuchElementException:
        #driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div._1YokD2._3Mn1Gg > div:nth-child(4) > div > div:nth-child(2) > button').click()
    #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//button[text()="Read More"]'))).click()
    
    try:
        phone=driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span').text
        smartphone_name.append(phone)
    except NoSuchElementException:
        smartphone_name.append("-")
    try:
      Colour=driver.find_element_by_xpath("//div[@class='_3k-BhJ']/table/tbody/tr[4]/td[2]/ul/li").text
      colour.append(Colour)  
    except NoSuchElementException:
        colour.append("-")
    try:
      RAM=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[4]/table/tbody/tr[2]/td[2]").text
      ram.append(RAM)
    except NoSuchElementException:
        ram.append("-")
        
    try:
      #
      #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
      ROM=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[4]/table/tbody/tr[1]/td[2]").text
      rom.append(ROM)
    except NoSuchElementException:
        rom.append("-")
        
    try:
      #
      #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
      prim=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[5]/table/tbody/tr[2]/td[2]").text
      primary_camera.append(prim)
    except NoSuchElementException:
        primary_camera.append("-")
        
    try:
        #
        #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
        sec=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[5]/table/tbody/tr[5]/td[2]").text
        secondary_camera.append(sec)
    except NoSuchElementException:
        secondary_camera.append("-")
        
    try:
        #
        #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
        ds=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[2]/table/tbody/tr[1]/td[2]").text
        display_size.append(ds)
    except NoSuchElementException:
        display_size.append("-")
        
    try:
        #
        #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
        drs=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[2]/table/tbody/tr[2]/td[2]").text
        display_resolution.append(drs)
    except NoSuchElementException:
        display_resolution.append("-")
        
    try:
          #
          #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
          pro_cessor=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[3]/table/tbody/tr[2]/td[2]").text
          processor.append(pro_cessor)
    except NoSuchElementException:
        processor.append("-")
        
    try:
        #
        #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
        pro_cessor_cores=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[3]/table/tbody/tr[3]/td[2]").text
        processor_cores.append(pro_cessor_cores)
    except NoSuchElementException:
        processor_cores.append("-")
        
    try:
        #driver.get(Phone_URL[i])
        #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
        batt=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[8]/table/tbody/tr[1]/td[2]").text
        battery_capacity.append(batt)
    except NoSuchElementException:
        battery_capacity.append("-")
        
    try:
        Price_=driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div:nth-child(2) > div > div.dyC4hf > div.CEmiEU > div > div').text
        price.append(Price_)
    except NoSuchElementException:
        price.append("-")
        
    time.sleep(5)

100%|██████████| 24/24 [02:56<00:00,  7.36s/it]


In [ ]:
smartphone_name

['ASUS ROG Phone 5 (Black, 128 GB)  (8 GB RAM)',
 'ASUS ROG Phone 3 (Black, 128 GB)  (12 GB RAM)',
 'ASUS ROG Phone 3 (Black, 128 GB)  (8 GB RAM)',
 'ASUS ZenFone 5Z (Midnight Blue, 128 GB)  (6 GB RAM)',
 'ASUS 6Z (Black, 64 GB)  (6 GB RAM)',
 'ASUS Zenfone 2 Laser ZE500KL (Black, 16 GB)  (2 GB RAM)',
 'ASUS ZenFone 5Z (Meteor Silver, 128 GB)  (6 GB RAM)',
 'ASUS ZenFone Max M2 (Silver, 64 GB)  (4 GB RAM)',
 'ASUS Zenfone Go (Silver, 8 GB)  (1 GB RAM)',
 'ASUS ZenFone Lite L1 (Black, 16 GB)  (2 GB RAM)',
 'ASUS Zenfone Go 5.5 (Gold, 32 GB)  (2 GB RAM)',
 'ASUS Zenfone Go (3rd Gen) (Gold, 16 GB)  (2 GB RAM)',
 'ASUS 6Z (Silver, 128 GB)  (6 GB RAM)',
 'ASUS ZenFone Max M2 (Blue, 32 GB)  (3 GB RAM)',
 'ASUS Zenfone Go (2nd Gen) (Red, 8 GB)  (1 GB RAM)',
 'ASUS ROG Phone II (Black, 128 GB)  (8 GB RAM)',
 'ASUS ROG Phone 5 (Black, 256 GB)  (12 GB RAM)',
 'ASUS ZenFone Max Pro M2 (Blue, 32 GB)  (3 GB RAM)',
 'ASUS Zenfone GO (Black, 16 GB)  (2 GB RAM)',
 'ASUS Zenfone Go 4.5 (Pink, 8 GB)  (1

In [ ]:
colour

['Black',
 'Black',
 'Black',
 'Midnight Blue',
 'Black',
 'Black',
 'Meteor Silver',
 'Silver',
 'Silver',
 'Black',
 'Gold',
 'Gold',
 'Silver',
 'Blue',
 'Red',
 'Black',
 'Black',
 'Blue',
 'Black',
 'Pink',
 'Gold',
 'Red',
 'Grey',
 'Silver']

In [ ]:
ram

['8 GB',
 '12 GB',
 '8 GB',
 '6 GB',
 '6 GB',
 'Qualcomm Snapdragon 410 MSM8916',
 '6 GB',
 '4 GB',
 '1 GB',
 '2 GB',
 '2 GB',
 '2 GB',
 '6 GB',
 '3 GB',
 '1 GB',
 '8 GB',
 '12 GB',
 '3 GB',
 '2 GB',
 '1 GB',
 '2 GB',
 '1 GB',
 '3 GB',
 '1 GB']

In [ ]:
rom

['128 GB',
 '128 GB',
 '128 GB',
 '128 GB',
 '64 GB',
 'Android Lollipop 5',
 '128 GB',
 '64 GB',
 '8 GB',
 '16 GB',
 '32 GB',
 '16 GB',
 '128 GB',
 '32 GB',
 '8 GB',
 '128 GB',
 '256 GB',
 '32 GB',
 '16 GB',
 '8 GB',
 '16 GB',
 '8 GB',
 '32 GB',
 '8 GB']

In [ ]:
primary_camera

['64MP + 13MP + 5MP',
 '64MP + 13MP + 5MP',
 '64MP + 13MP + 5MP',
 '12MP + 8MP',
 '48MP + 13MP',
 '2 GB',
 '12MP + 8MP',
 '13MP + 2MP',
 '5MP Rear Camera',
 '13MP Rear Camera',
 'Yes',
 '13MP Rear Camera',
 '48MP + 13MP',
 '13MP + 2MP',
 '5MP Rear Camera',
 '48MP + 13MP',
 '64MP + 13MP + 5MP',
 '12MP + 5MP',
 '13MP Rear Camera',
 '5MP Rear Camera',
 '13MP Rear Camera',
 '5MP Rear Camera',
 '13MP Rear Camera',
 '8MP Rear Camera']

In [ ]:
secondary_camera

['24MP Front Camera',
 'Yes',
 'Yes',
 '8MP Front Camera',
 '48MP + 13MP Dual Front Camera',
 'Yes',
 '8MP Front Camera',
 '8MP Front Camera',
 '0.3MP Front Camera',
 '5MP Front Camera',
 '-',
 '5MP Front Camera',
 '48MP + 13MP Dual Front Camera',
 '8MP Front Camera',
 '0.3MP Front Camera',
 'Yes',
 '24MP Front Camera',
 '13MP Front Camera',
 '5MP Front Camera',
 '0.3MP Front Camera',
 'Yes',
 '0.3MP Front Camera',
 '13MP Front Camera',
 '2MP Front Camera']

In [ ]:
display_size

['17.22 cm (6.78 inch)',
 '16.74 cm (6.59 inch)',
 '16.74 cm (6.59 inch)',
 '15.75 cm (6.2 inch)',
 '16.23 cm (6.39 inch)',
 'Handset, Standard USB Cable, Adapter, User Manual, Warranty Card',
 '15.75 cm (6.2 inch)',
 '15.9 cm (6.26 inch)',
 '11.43 cm (4.5 inch)',
 '13.84 cm (5.45 inch)',
 '13.97 cm (5.5 inch)',
 '13.97 cm (5.5 inch)',
 '16.23 cm (6.39 inch)',
 '15.9 cm (6.26 inch)',
 '11.43 cm (4.5 inch)',
 '16.74 cm (6.59 inch)',
 '17.22 cm (6.78 inch)',
 '15.9 cm (6.26 inch)',
 '13.97 cm (5.5 inch)',
 '11.43 cm (4.5 inch)',
 '12.7 cm (5 inch)',
 '11.43 cm (4.5 inch)',
 '13.97 cm (5.5 inch)',
 '11.43 cm (4.5 inch)']

In [ ]:
display_resolution

['2448 x 1080 Pixels',
 '2340 x 1080 Pixels',
 '2340 x 1080 Pixels',
 '2246 x 1080 Pixels',
 '2340 x 1080 pixels',
 'ZE500KL-1A079WW/ ZE500KL-1A246IN',
 '2246 x 1080 Pixels',
 '1520 x 720 pixels',
 '854 x 480 Pixels',
 '1440 x 720 pixels',
 '1280 x 720 Pixels',
 '1280 x 720 Pixels',
 '2340 x 1080 pixels',
 '1520 x 720 pixels',
 '854 x 480 Pixels',
 '2340 x 1080 Pixels',
 '2448 x 1080 Pixels',
 '2280 x 1080 pixels',
 '1280 x 720',
 '480 x 854 Pixels',
 '1280 x 720 Pixels',
 '854 x 480 Pixels',
 '1920 x 1080 Pixels',
 '854 x 480 Pixels']

In [ ]:
processor

['Qualcomm Snapdragon 888 (SM8350)',
 'Qualcomm Snapdragon 865+ (SM8250)',
 'Qualcomm Snapdragon 865+ (SM8250)',
 'Qualcomm Snapdragon 845 Octa Core',
 'Qualcomm SD 855',
 '1280 x 720 Pixels',
 'Qualcomm Snapdragon 845 Octa Core',
 'Qualcomm Snapdragon 632 Octa Core',
 'Qualcomm Snapdragon 200',
 'Qualcomm Snapdragon 430',
 'Quad Core',
 'Qualcomm Snapdradon 410 (MSM8916)',
 'Qualcomm SD 855',
 'Qualcomm Snapdragon 632 Octa Core',
 'Qualcomm Snapdragon 200',
 'Qualcomm Snapdragon 855 Plus',
 'Qualcomm Snapdragon 888 (SM8350)',
 'Qualcomm Snapdragon 660 Octa Core',
 'Quad Core',
 'MediaTek MT6580M',
 'Quad Core',
 'Intel Atom CloverTrail Z2520',
 'Qualcomm Snapdragon 615 MSM8939',
 'Qualcomm Snapdragon 200']

In [ ]:
processor_cores

['Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'HD',
 'Octa Core',
 'Octa Core',
 'Quad Core',
 'Octa Core',
 '1.4 GHz',
 'Quad Core',
 'Octa Core',
 'Octa Core',
 'Quad Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 '1.2 GHz',
 'Quad Core',
 '1.2 GHz',
 'Quad Core',
 'Octa Core',
 'Quad Core']

In [ ]:
battery_capacity

['6000 mAh',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 '3G, 4G, 2G',
 'Yes',
 'Yes',
 'Yes',
 '3000 mAh',
 '3010 mAh',
 '3000 mAh',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 '6000 mAh',
 'Yes',
 'Yes',
 'Yes',
 'Capacitive',
 'Yes',
 'Yes',
 'Yes']

In [ ]:
price

['₹15,599',
 '₹36,299',
 '₹7,999',
 '₹14,999',
 '₹14,999',
 '₹5,299',
 '₹36,299',
 '₹5,299',
 '₹7,199',
 '₹6,999',
 '₹8,999',
 '₹12,999',
 '₹12,999',
 '₹5,299',
 '₹15,999',
 '₹7,199',
 '₹7,999',
 '₹5,299',
 '₹17,999',
 '₹5,699',
 '₹5,699',
 '₹6,999',
 '₹8,499',
 '₹14,999']

In [ ]:
#extracting brand_name from smartphone_name
brand_name = [i.split(maxsplit=1)[0] for i in smartphone_name]

In [ ]:
brand_name

['ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS',
 'ASUS']

In [ ]:
#printing lengths
print(len(brand_name),len(smartphone_name),len(colour),len(ram),len(rom),len(primary_camera),len(secondary_camera),len(display_size),len(display_resolution),len(processor),len(processor_cores),len(battery_capacity),len(price),len(Phone_URL))

24 24 24 24 24 24 24 24 24 24 24 24 24 24


In [ ]:
#creating a dataframe

Smartphone=pd.DataFrame({})
Smartphone['brand_name']=brand_name
Smartphone['smartphone_name']=smartphone_name
Smartphone['colour']=colour
Smartphone['Ram']=ram
Smartphone['Rom']=rom
Smartphone['primary_camera']=primary_camera
Smartphone['secondary_camera']=secondary_camera
Smartphone['display_size']=display_size
Smartphone['display_resolution']=display_resolution
Smartphone['processor']=processor
Smartphone['processor_cores']=processor_cores
Smartphone['battery_capacity']=battery_capacity
Smartphone['price']=price
Smartphone['Phone_URL']=Phone_URL

In [ ]:
Smartphone

,brand_name,smartphone_name,colour,Ram,Rom,primary_camera,secondary_camera,display_size,display_resolution,processor,processor_cores,battery_capacity,price,Phone_URL
0,ASUS,"ASUS ROG Phone 5 (Black, 128 GB) (8 GB RAM)",Black,8 GB,128 GB,64MP + 13MP + 5MP,24MP Front Camera,17.22 cm (6.78 inch),2448 x 1080 Pixels,Qualcomm Snapdragon 888 (SM8350),Octa Core,6000 mAh,"₹49,999",https://www.flipkart.com/asus-rog-phone-5-blac...
1,ASUS,"ASUS ROG Phone 3 (Black, 128 GB) (12 GB RAM)",Black,12 GB,128 GB,64MP + 13MP + 5MP,Yes,16.74 cm (6.59 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 865+ (SM8250),Octa Core,Yes,"₹49,999",https://www.flipkart.com/asus-rog-phone-3-blac...
2,ASUS,"ASUS ROG Phone 3 (Black, 128 GB) (8 GB RAM)",Black,8 GB,128 GB,64MP + 13MP + 5MP,Yes,16.74 cm (6.59 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 865+ (SM8250),Octa Core,Yes,"₹46,999",https://www.flipkart.com/asus-rog-phone-3-blac...
3,ASUS,"ASUS ZenFone 5Z (Midnight Blue, 128 GB) (6 GB...",Midnight Blue,6 GB,128 GB,12MP + 8MP,8MP Front Camera,15.75 cm (6.2 inch),2246 x 1080 Pixels,Qualcomm Snapdragon 845 Octa Core,Octa Core,Yes,"₹36,299",https://www.flipkart.com/asus-zenfone-5z-midni...
4,ASUS,"ASUS 6Z (Black, 64 GB) (6 GB RAM)",Black,6 GB,64 GB,48MP + 13MP,48MP + 13MP Dual Front Camera,16.23 cm (6.39 inch),2340 x 1080 pixels,Qualcomm SD 855,Octa Core,Yes,"₹35,999",https://www.flipkart.com/asus-6z-black-64-gb/p...
5,ASUS,"ASUS Zenfone 2 Laser ZE500KL (Black, 16 GB) (...",Black,Qualcomm Snapdragon 410 MSM8916,Android Lollipop 5,2 GB,Yes,"Handset, Standard USB Cable, Adapter, User Man...",ZE500KL-1A079WW/ ZE500KL-1A246IN,1280 x 720 Pixels,HD,"3G, 4G, 2G","₹7,999",https://www.flipkart.com/asus-zenfone-2-laser-...
6,ASUS,"ASUS ZenFone 5Z (Meteor Silver, 128 GB) (6 GB...",Meteor Silver,6 GB,128 GB,12MP + 8MP,8MP Front Camera,15.75 cm (6.2 inch),2246 x 1080 Pixels,Qualcomm Snapdragon 845 Octa Core,Octa Core,Yes,"₹36,299",https://www.flipkart.com/asus-zenfone-5z-meteo...
7,ASUS,"ASUS ZenFone Max M2 (Silver, 64 GB) (4 GB RAM)",Silver,4 GB,64 GB,13MP + 2MP,8MP Front Camera,15.9 cm (6.26 inch),1520 x 720 pixels,Qualcomm Snapdragon 632 Octa Core,Octa Core,Yes,"₹14,999",https://www.flipkart.com/asus-zenfone-max-m2-s...
8,ASUS,"ASUS Zenfone Go (Silver, 8 GB) (1 GB RAM)",Silver,1 GB,8 GB,5MP Rear Camera,0.3MP Front Camera,11.43 cm (4.5 inch),854 x 480 Pixels,Qualcomm Snapdragon 200,Quad Core,Yes,"₹5,299",https://www.flipkart.com/asus-zenfone-go-silve...
9,ASUS,"ASUS ZenFone Lite L1 (Black, 16 GB) (2 GB RAM)",Black,2 GB,16 GB,13MP Rear Camera,5MP Front Camera,13.84 cm (5.45 inch),1440 x 720 pixels,Qualcomm Snapdragon 430,Octa Core,3000 mAh,"₹7,199",https://www.flipkart.com/asus-zenfone-lite-l1-...


In [ ]:
#converting into csv format
Smartphone.to_csv("Smartphone.csv")

In [ ]:
#closing the driver
driver.close()

### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [ ]:
import re
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver= webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
url='https://www.google.co.in/maps'
driver.get(url)
time.sleep(3)

In [ ]:
city = input('Enter City Name : ')                                         
search = driver.find_element_by_id("searchboxinput")                       
search.clear()                                                             
time.sleep(3)
search.send_keys(city)                                                     
button = driver.find_element_by_id("searchbox-searchbutton")               
button.click()                                                             
time.sleep(3)

Enter City Name : Silvassa


In [ ]:
try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

URL Extracted:  https://www.google.co.in/maps/place/Silvassa,+Dadra+and+Nagar+Haveli+and+Daman+and+Diu,+India/@20.2665228,72.970225,13z/data=!3m1!4b1!4m5!3m4!1s0x3be0cb5edb1ce063:0x65fe78dc1cdaeb1f!8m2!3d20.2753406!4d73.0072986
Latitude = 20.2665228, Longitude = 72.970225


### 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver= webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [ ]:
url='https://trak.in/'

In [ ]:
driver.get(url)

In [ ]:
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [ ]:
Date=[]
Startup=[]
Industry=[]
SubVertical=[]
Location=[]
Investor=[]
Investment=[]
Amount=[]

In [ ]:
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        Date.append(d.text)

    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        Startup.append(n.text)
    
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        Industry.append(n.text)
    
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        SubVertical.append(s.text)

    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        Location.append(l.text)
    
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        Investor.append(n.text)
    
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        Investment.append(n.text)
    
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        Amount.append(a.text)

In [ ]:
fund=pd.DataFrame({})
fund['Date']=Date
fund['Startup']=Startup 
fund['Industry']=Industry
fund['SubVertical']=SubVertical
fund['Location']=Location
fund['Investor']=Investor
fund['Investment']=Investment
fund['Amount']=Amount

In [ ]:
fund

,Date,Startup,Industry,SubVertical,Location,Investor,Investment,Amount
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


### 7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in. 

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver= webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [ ]:
url = "https://www.digit.in/"
driver.get(url)
time.sleep(3)
#lets search for best laptop
best_gaming_lap = driver.find_element_by_xpath("//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)

In [ ]:
#lets fetch info of laptops
lap_name = []
op_stm = []
display = []
processor = []
memory = []
weight = []
dimensions = []
graph_processor = []
prices = []
# lets fetch laptop names
name = driver.find_elements_by_xpath("//table[@id='summtable']//tr//td[1]")
for i in name:
    lap_name.append(i.text)
#fetching info related to operating system
try:
    os = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[3]//td[3]")
    for i in os:
        op_stm.append(i.text)
except NoSuchElementException:
    pass
try:
    disp = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[4]//td[3]")
    for i in disp:
        display.append(i.text)
except NoSuchElementException:
    pass
try:
    pro = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[5]//td[3]")
    for i in pro:
        processor.append(i.text)
except NoSuchElementException:
    pass
try:
    mem = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[6]//td[3]")
    for i in mem:
        memory.append(i.text)
except NoSuchElementException:
    pass
try:
    wgt = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[7]//td[3]")
    for i in wgt:
        weight.append(i.text)
except NoSuchElementException:
    pass
try:
    dim = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[8]//td[3]")
    for i in dim:
        dimensions.append(i.text)
except NoSuchElementException:
    pass
try:
    g_pro = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[9]//td[3]")
    for i in g_pro:
        graph_processor.append(i.text)
except NoSuchElementException:
    pass
try:
    price = driver.find_elements_by_xpath("//td[@class='smprice']")
    for i in price:
        prices.append(i.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

#creating dataframe


In [ ]:
data = list(zip(lap_name,op_stm,display,processor,memory,weight,dimensions,graph_processor,prices))
df = pd.DataFrame(data, columns = ["Laptop Name","Operating System","Display","Processor","Memory","Weight(Kg)","Dimensions","Graphic-Processor","Price"])
df

,Laptop Name,Operating System,Display,Processor,Memory,Weight(Kg),Dimensions,Graphic-Processor,Price
0,Alienware Area 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,N/A
1,Alienware m15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,"Rs 341,990"
2,ASUS ROG Strix Scar 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,N/A
3,Asus ROG Zephyrus G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"Rs 164,990"
4,Lenovo Legion 5i,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,"Rs 76,988"
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"Rs 185,000"
6,Acer Aspire 7 gaming,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,"Rs 64,370"


In [ ]:
#closing the driver
driver.close()

## Q.NO.8 Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver= webdriver.Chrome('chromedriver',chrome_options=chrome_options)
#driver = webdriver.Chrome("chromedriver.exe")
#get the web page with given url
url = "https://www.forbes.com/?sh=55783e032254"
driver.get(url)
time.sleep(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [ ]:
driver.find_element_by_xpath("//div[@class='header__left']//button").click()

#Now select Billionaires from options



In [ ]:
blns = driver.find_elements_by_xpath("//a[@class='section__link']")

In [ ]:
bln = []
for i in blns[:1]:
    bln.append(i.get_attribute("href"))

In [ ]:
bln

['https://www.forbes.com/billionaires/']

In [ ]:
for i in bln:
    driver.get(i)
# driver.find_element_by_xpath("//div[@class='channel-special-feature__articles-wrap']//a").click()
# time.sleep(8)

In [ ]:
rank = []
try:
    rnk = driver.find_elements_by_xpath("//div[@class='table-row-group']/div/div/div/div")
    for i in rnk:
        rank.append(i.text)
except NoSuchElementException:
    pass

In [ ]:
rank

['',
 'Amazon',
 '',
 'Jeff Bezos founded e-commerce giant Amazon in 1994 out of his garage in Seattle. He stepped down as CEO to become executive chairman on July 5, 2021.',
 'FULL PROFILE',
 '',
 'Tesla, SpaceX',
 '',
 '',
 'LVMH',
 '',
 '',
 'Microsoft',
 '',
 '',
 'Facebook',
 '',
 '',
 'Berkshire Hathaway',
 '',
 '',
 'software',
 '',
 '',
 'Google',
 '',
 '',
 'Google',
 '',
 '',
 'diversified',
 '',
 '',
 'Zara',
 '',
 '',
 "L'Oréal",
 '',
 '',
 'beverages, pharmaceuticals',
 '',
 '',
 'Microsoft',
 '',
 '',
 'internet media',
 '',
 '',
 'telecom',
 '',
 '',
 'Walmart',
 '',
 '',
 'Walmart',
 '',
 '',
 'Walmart',
 '',
 '',
 'Bloomberg LP',
 '',
 '',
 'e-commerce',
 '',
 '',
 'Amazon',
 '',
 '',
 'Quicken Loans',
 '',
 '',
 'infrastructure, commodities',
 '',
 '',
 'Nike',
 '',
 '',
 'e-commerce',
 '',
 '',
 'Koch Industries',
 '',
 '',
 'Koch Industries',
 '',
 '',
 'internet, telecom',
 '',
 '',
 'Dell computers',
 '',
 '',
 'fashion retail',
 '',
 '',
 'luxury goods',
 '',
 ''

In [ ]:
#lets get option button from the page
driver.find_element_by_xpath("//div[@class='header__left']//button").click()

#Now select Billionaires from options
blns = driver.find_elements_by_xpath("//a[@class='header__title']")
bln = []
for i in blns[:1]:
    bln.append(i.get_attribute("href"))
     

for i in bln:
    driver.get(i)
#driver.find_element_by_xpath("//div[@class='channel-special-feature__articles-wrap']//a").click()
time.sleep(8)

#fetch required data from table
#fetching rank
rank = []
try:
    rnk = driver.find_elements_by_xpath("//div[@class='rank']")
    for i in rnk:
        rank.append(i.text)
except NoSuchElementException:
    pass
#fetching names
names = []
try:
    name = driver.find_elements_by_xpath("//div[@class='personName']")
    for i in name:
        names.append(i.text)
except NoSuchElementException:
    pass
#fetching net worth
net_worth = []
try:
    worth = driver.find_elements_by_xpath("//div[@class='netWorth']")
    for i in worth:
        net_worth.append(i.text)
except NoSuchElementException:
    pass
#fetching age
Age = []
try:
    age = driver.find_elements_by_xpath("//div[@class='age']/div")
    for i in age:
        Age.append(i.text)
except NoSuchElementException:
    Age.append('-')
#fetching citizenship
citizenship = []
try:
    cit = driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for i in cit:
        citizenship.append(i.text)
except NoSuchElementException:
    citizenship.append('-')
#fetching source
source = []
try:
    src = driver.find_elements_by_xpath("//div[@class='source-column']")
    for i in src:
        source.append(i.text)
except NoSuchElementException:
    source.append('-')
#fetching Industry
industry = []
try:
    ind = driver.find_elements_by_xpath("//div[@class='category']")
    for i in ind:
        industry.append(i.text)
except NoSuchElementException:
    industry.append('-')
    
#creating dataframe
data = list(zip(rank,names,net_worth,Age,citizenship,source,industry))
df = pd.DataFrame(data, columns = ["Rank", "Name", "Net worth", "Age", "Citizenship", "Source","Industry"])
df